In [1]:
import pandas as pd
import requests
from datetime import datetime
import os


In [2]:
def remove_multiindex(df:pd.DataFrame):
	return df.droplevel(level=0,axis=1)

In [3]:
# link for extract html data
def getdata(url):
    r = requests.get(url)
    return r.text

def get_nba_season_year(date=None):
    if date is None:
        date = datetime.now()
    return date.year + 1 if date.month >= 10 else date.year

In [5]:
full_df_list = pd.DataFrame(columns=[ 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
		   '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
		   'BLK', 'TOV', 'PF', 'PTS', 'Age', 'W', 'L', 'MOV',
		   'SOS', 'SRS', 'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%',])


for year in range(1980,get_nba_season_year() + 1):
	print('Fetching',year)

	if os.path.exists(f'team_stats/{year}_tm_stats.csv') and year != get_nba_season_year():
		new = pd.read_csv(f'team_stats/{year}_tm_stats.csv')
	else:

		htmldata = getdata(f"https://www.basketball-reference.com/leagues/NBA_{year}.html")
		df_list = pd.read_html(htmldata)

		if year <= 2015:
			per_game_stat = df_list[3].copy()

			adv_stat = df_list[8].copy()
		else:
			per_game_stat = df_list[4].copy()

			adv_stat = df_list[10].copy()

		new = pd.merge(per_game_stat,adv_stat.droplevel(level=0,axis=1),on='Team')

		new = new[[ 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
		   '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
		   'BLK', 'TOV', 'PF', 'PTS', 'Age', 'W', 'L', 'MOV',
		   'SOS', 'SRS', 'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%',]].sort_values('W',ascending=False).reset_index(drop=True)

		new['Year'] = year

		new.loc[new['Team'] == 'League Average','Team'] = f'League Average {year}'

		new.to_csv(f'team_stats/{year}_tm_stats.csv')
	full_df_list = pd.concat([full_df_list,new])

	print(year,'Complete')


full_df_list.to_csv('team_stats/all.csv')

In [9]:
full_df_list.to_csv('../Data/Team Data/Combined/league_stats.csv')